In [1]:
!pip install rasterio geojson rasterstats owslib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 61.1 MB/s eta 0:00:00


In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
from owslib.wcs import WebCoverageService
import requests
import geojson
import rasterio
import geopandas as gpd
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from rasterstats import zonal_stats
from io import BytesIO
from shapely.geometry import box
from matplotlib.colors import ListedColormap, BoundaryNorm


## Importando camada vetorial

In [3]:
# Parâmetros para conectar com a camada vetorial
mun_url = "https://info.dengue.mat.br/geoserver/wfs"

params_AML = dict(
    service="WFS",
    version="2.0.0",
    request="GetFeature",
    typeName="AP_subdistritos_CD2022", # usando camada de subdistritos, pois municípios na Amazônia são gigantes
    outputFormat="json",
)

# Fazendo o request
r_AML = requests.get(mun_url, params=params_AML)

# Baixando e carregando a camada
municipios = gpd.GeoDataFrame.from_features(geojson.loads(r_AML.content), crs="EPSG:4674")


In [4]:
municipios.head()

,geometry,id,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,SIGLA
0,"POLYGON ((-52.00182 0.93616, -52.00186 0.93637...",1,1,Norte,16,Amapá,1600055,Serra do Navio,160005505,Serra do Navio,16000550500,None,1602,Oiapoque - Porto Grande,160004,Porto Grande,None,None,AP
1,"POLYGON ((-52.01806 0.88724, -52.01771 0.8863,...",2,1,Norte,16,Amapá,1600055,Serra do Navio,160005510,Cachaço,16000551000,None,1602,Oiapoque - Porto Grande,160004,Porto Grande,None,None,AP
2,"POLYGON ((-50.45011 2.10924, -50.44715 2.10917...",3,1,Norte,16,Amapá,1600105,Amapá,160010505,Amapá,16001050500,None,1602,Oiapoque - Porto Grande,160003,Oiapoque,None,None,AP
3,"POLYGON ((-50.5088 2.18518, -50.5069 2.18443, ...",4,1,Norte,16,Amapá,1600105,Amapá,160010505,Amapá,16001050500,None,1602,Oiapoque - Porto Grande,160003,Oiapoque,None,None,AP
4,"POLYGON ((-50.77464 1.83875, -50.7747 1.8383, ...",5,1,Norte,16,Amapá,1600105,Amapá,160010505,Amapá,16001050500,None,1602,Oiapoque - Porto Grande,160003,Oiapoque,None,None,AP


In [5]:
# Filtrar para o município de estudo
municipio_estudo = "Oiapoque"

municipio_bbox = municipios[municipios['NM_DIST'].str.contains(municipio_estudo, case=False)]

# Obter os limites (bounding box) do Oiapoque/AP
xmin, ymin, xmax, ymax = municipio_bbox.total_bounds
xmin, ymin, xmax, ymax


(np.float64(-51.855426),
 np.float64(3.17639),
 np.float64(-51.356847),
 np.float64(4.394141))

In [6]:
municipio_bbox.total_bounds

array([-51.855426,   3.17639 , -51.356847,   4.394141])

## Importando rasters de 1985 e 2023

### 1 - Descobrindo o tamanho em pixels do raster

In [7]:
import requests
import xml.etree.ElementTree as ET

# Parâmetros para o WCS e a cobertura
wcs_url = "https://info.dengue.mat.br/geoserver/wcs"
wcs_version = "1.0.0"
coverage_id = "brasil_uso_cob:mapbiomas_brasil_coverage_1985"

# Monta a URL para o DescribeCoverage (sem espaços, com os parâmetros necessários)
describe_url = (f"{wcs_url}?service=WCS&version={wcs_version}"
                f"&request=DescribeCoverage&coverage={coverage_id}")

r_describe = requests.get(describe_url)
if r_describe.status_code != 200:
    raise Exception("Erro no DescribeCoverage: " + str(r_describe.status_code))

# Parseia o XML da resposta
root = ET.fromstring(r_describe.content)
# Definir os namespaces – pode ser necessário ajustar se o XML usar outros valores
ns = {
    "wcs": "http://www.opengis.net/wcs",
    "gml": "http://www.opengis.net/gml"
}

# Procura o elemento GridEnvelope na resposta (normalmente dentro de CoverageDescription)
grid_env = root.find(".//gml:GridEnvelope", ns)
if grid_env is None:
    raise Exception("Nenhum elemento <gml:GridEnvelope> encontrado no DescribeCoverage.")

low_elem = grid_env.find("gml:low", ns)
high_elem = grid_env.find("gml:high", ns)
if low_elem is None or high_elem is None:
    raise Exception("Elementos <gml:low> ou <gml:high> não foram encontrados no GridEnvelope.")

# Os valores geralmente vêm como uma string com dois números separados por espaço
low_vals = list(map(int, low_elem.text.split()))
high_vals = list(map(int, high_elem.text.split()))
native_width = high_vals[0] - low_vals[0] + 1
native_height = high_vals[1] - low_vals[1] + 1
print("Dimensões nativas extraídas (width x height):", native_width, "x", native_height)


Dimensões nativas extraídas (width x height): 155241 x 158828


### 2 - Calculando o tamanho em pixels do recorte que faremos (para manter a mesma resolucão)

In [8]:
# Metadados do raster original (exemplo)
orig_bbox = (-73.98318216, -16.66197917, -43.39929216, 5.26958083)  # (xmin, ymin, xmax, ymax)
native_width = 155241   # largura original em pixels
native_height = 158828  # altura original em pixels

# Calcular resolução em x e y
res_x = (orig_bbox[2] - orig_bbox[0]) / native_width
res_y = (orig_bbox[3] - orig_bbox[1]) / native_height
print("Resolução em x:", res_x)
print("Resolução em y:", res_y)

# Definir a bbox do recorte (por exemplo, uma região de interesse)
crop_bbox = (float(xmin), float(ymin), float(xmax), float(ymax))  # convertendo de np.float para float simples
# crop_bbox = (-52.7506, 3.6626, -51.7776, 4.0039)

# Calcular dimensões do recorte em pixels
crop_width_pixels = int(round((crop_bbox[2] - crop_bbox[0]) / res_x))
crop_height_pixels = int(round((crop_bbox[3] - crop_bbox[1]) / res_y))

print("Dimensões nativas do recorte (pixels): {} x {}".format(crop_width_pixels, crop_height_pixels))


Resolução em x: 0.00019700910197692617
Resolução em y: 0.000138083713199184
Dimensões nativas do recorte (pixels): 2531 x 8819


In [9]:
crop_bbox

(-51.855426, 3.17639, -51.356847, 4.394141)

In [10]:
# Lista de anos com Terraclass disponível
years = [1985, 1986, 1987, 1988, 1989, 1990]
crs = "EPSG:4674"
output_format = "image/geotiff"

wcs = WebCoverageService(wcs_url, version=wcs_version, timeout=None)

for year in years:
  coverage_id = f"brasil_uso_cob:mapbiomas_brasil_coverage_{year}"
  output_file = f"mapbiomas_coverage_{year}.tif"

  print(f"requisitando cobertura {coverage_id}")

  response = wcs.getCoverage(
      identifier=coverage_id,
      format=output_format,
      crs=crs,
      bbox=crop_bbox,
      width=crop_width_pixels,
      height=crop_height_pixels,
      timeout=None
      )

  with open(output_file, "wb") as f:
    f.write(response.read())

  print(f"{output_file} salvo com sucesso")


requisitando cobertura brasil_uso_cob:mapbiomas_brasil_coverage_1985
mapbiomas_coverage_1985.tif salvo com sucesso
requisitando cobertura brasil_uso_cob:mapbiomas_brasil_coverage_1986
mapbiomas_coverage_1986.tif salvo com sucesso
requisitando cobertura brasil_uso_cob:mapbiomas_brasil_coverage_1987
mapbiomas_coverage_1987.tif salvo com sucesso
requisitando cobertura brasil_uso_cob:mapbiomas_brasil_coverage_1988
mapbiomas_coverage_1988.tif salvo com sucesso
requisitando cobertura brasil_uso_cob:mapbiomas_brasil_coverage_1989
mapbiomas_coverage_1989.tif salvo com sucesso
requisitando cobertura brasil_uso_cob:mapbiomas_brasil_coverage_1990
mapbiomas_coverage_1990.tif salvo com sucesso


In [11]:
src_1985 = rasterio.open("mapbiomas_coverage_1985.tif")
src_1986 = rasterio.open("mapbiomas_coverage_1986.tif")
src_1987 = rasterio.open("mapbiomas_coverage_1987.tif")
src_1988 = rasterio.open("mapbiomas_coverage_1988.tif")
src_1989 = rasterio.open("mapbiomas_coverage_1989.tif")
src_1990 = rasterio.open("mapbiomas_coverage_1990.tif")


In [12]:
years = [1985, 1986, 1987, 1988, 1989, 1990]

data = {}
profiles = {}
nodata_vals = {}
transforms = {}

for year in years:
  path = f"mapbiomas_coverage_{year}.tif"
  print(f"Lendo {path}…")
  with rasterio.open(path) as src:
    # lê banda 1 como array NumPy
    data[year] = src.read(1)
    # metadados (dimensão, CRS, dtype, etc.)
    profiles[year] = src.profile
    # valor de nodata
    nodata_vals[year] = src.nodata
    # transform affine
    transforms[year] = src.transform


Lendo mapbiomas_coverage_1985.tif…
Lendo mapbiomas_coverage_1986.tif…
Lendo mapbiomas_coverage_1987.tif…
Lendo mapbiomas_coverage_1988.tif…
Lendo mapbiomas_coverage_1989.tif…
Lendo mapbiomas_coverage_1990.tif…


In [13]:
data_1985 = data[1985]
data_1986 = data[1986]
data_1987 = data[1987]
data_1988 = data[1988]
data_1989 = data[1989]
data_1990 = data[1989]

In [14]:
profile_1985 = profiles[1985]
profile_1986 = profiles[1986]
profile_1987 = profiles[1987]
profile_1988 = profiles[1988]
profile_1989 = profiles[1989]
profile_1990 = profiles[1990]


In [15]:
nodata_1985 = nodata_vals[1985]
nodata_1986 = nodata_vals[1986]
nodata_1987 = nodata_vals[1987]
nodata_1988 = nodata_vals[1988]
nodata_1989 = nodata_vals[1989]
nodata_1990 = nodata_vals[1990]


In [16]:
transform_1985 = transforms[1985]
transform_1986 = transforms[1986]
transform_1987 = transforms[1987]
transform_1988 = transforms[1988]
transform_1989 = transforms[1989]
transform_1990 = transforms[1990]


In [17]:
# Se os rasters tiverem valor NoData, mascaramos esses pixels para que apareçam em branco
if nodata_1985 is not None:
    data_1985 = np.ma.masked_equal(data_1985, nodata_1985)
if nodata_1986 is not None:
    data_1986 = np.ma.masked_equal(data_1986, nodata_1986)
if nodata_1987 is not None:
    data_1987 = np.ma.masked_equal(data_1987, nodata_1987)
if nodata_1988 is not None:
    data_1988 = np.ma.masked_equal(data_1988, nodata_1988)
if nodata_1989 is not None:
    data_1989 = np.ma.masked_equal(data_1989, nodata_1989)
if nodata_1990 is not None:
    data_1990 = np.ma.masked_equal(data_1990, nodata_1990)


In [18]:
extent = [src_1985.bounds.left, src_1985.bounds.right, src_1985.bounds.bottom, src_1985.bounds.top]
extent


[-51.855426, -51.35684700000001, 3.1763900001129484, 4.394141000297353]

## Extraindo estatísticas zonais

In [19]:
municipio_estudo = "Oiapoque"

mun_estudo = municipios[municipios['NM_DIST'].str.contains(municipio_estudo, case=False)]


In [20]:
years = [1985, 1986, 1987, 1988, 1989, 1990]
zs = {}

for year in years:
  zs[year] = zonal_stats(mun_estudo, data[year], affine=transforms[year], categorical=True, nodata=nodata_vals[year])


In [21]:
zs[1985]

[{3: 4957636,
  4: 202,
  5: 5311,
  6: 2247848,
  11: 2851511,
  12: 66652,
  15: 15228,
  24: 431,
  33: 307142}]

In [22]:
# extraímos o único elemento de cada lista e montamos um dict “ano = {classe: contagem, …}”
flat = {year: zs[year][0] for year in years}

# criamos o DataFrame, transpondo para que cada linha seja um ano
df = pd.DataFrame.from_dict(flat, orient='index').fillna(0)

# converte o índice (ano) em coluna
df.index.name = 'year'
df.reset_index(inplace=True)

print(df)


   year        3    4     5        6       11     12     15   24      33
0  1985  4957636  202  5311  2247848  2851511  66652  15228  431  307142
1  1986  4958121  202  5311  2249945  2795023  62119  73648  514  307088
2  1987  4958118  202  5311  2246993  2797946  62117  73646  546  307092
3  1988  4955483  286  5311  2246716  2794862  62325  79354  546  307088
4  1989  4955484  285  5311  2246715  2794894  62176  79472  546  307088
5  1990  4955484  285  5311  2246715  2794894  62176  79472  546  307088


In [23]:
# Dicionário de mapeamento
palette = {
    3: "Forest Formation",
    4: "Savanna Formation",
    5: "Mangrove",
    6: "Floodable Forest",
    9: "Forest Plantation",
    11: "Wetland",
    12: "Grassland",
    15: "Pasture",
    20: "Sugar Cane",
    21: "Mosaic of Uses",
    23: "Beach, Dune and Sand Spot",
    24: "Urban Area",
    25: "Other non Vegetated Areas",
    29: "Rocky Outcrop",
    30: "Mining",
    31: "Aquaculture",
    32: "Hypersaline Tidal Flat",
    33: "River, Lake and Ocean",
    35: "Palm Oil",
    39: "Soybean",
    40: "Rice",
    41: "Other Temporary Crops",
    46: "Coffee",
    47: "Citrus",
    48: "Other Perennial Crops",
    49: "Wooded Sandbank Vegetation",
}

# Monta um dicionário de renomeação só com as colunas que existem em df
col_mapping = {}
for k, v in palette.items():
    if k in df.columns:
        col_mapping[k] = v
    elif str(k) in df.columns:
        col_mapping[str(k)] = v

# Aplica o renomeamento
df_final = df.rename(columns=col_mapping)

print(df_final.columns)


Index(['year', 'Forest Formation', 'Savanna Formation', 'Mangrove',
       'Floodable Forest', 'Wetland', 'Grassland', 'Pasture', 'Urban Area',
       'River, Lake and Ocean'],
      dtype='object')


In [24]:
print(df_final)


   year  Forest Formation  Savanna Formation  Mangrove  Floodable Forest  \
0  1985           4957636                202      5311           2247848   
1  1986           4958121                202      5311           2249945   
2  1987           4958118                202      5311           2246993   
3  1988           4955483                286      5311           2246716   
4  1989           4955484                285      5311           2246715   
5  1990           4955484                285      5311           2246715   

   Wetland  Grassland  Pasture  Urban Area  River, Lake and Ocean  
0  2851511      66652    15228         431                 307142  
1  2795023      62119    73648         514                 307088  
2  2797946      62117    73646         546                 307092  
3  2794862      62325    79354         546                 307088  
4  2794894      62176    79472         546                 307088  
5  2794894      62176    79472         546                 